In [ ]:
import os
import re
import time
import random
import math
import requests
import json
import pandas as pd
from tqdm import tqdm
from dotenv import load_dotenv
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

In [ ]:
# 读取需要的环境变量
load_dotenv()
# link info
SF_URL = os.getenv('SF_URL')
LDTJ_URL = os.getenv('LDTJ_URL')
JBXX_URL = os.getenv('JBXX_URL')
CTD_URL = os.getenv('CTD_URL')
IMG_URL = os.getenv('IMG_URL')
PM_URL = os.getenv('PM_URL')
PM_FC_URL = os.getenv('PM_FC_URL')
DSH_URL = 'https://sanpu.iarrp.cn/api/ssp/dcyd/xj/dzjsh/sl?xjdm=520327'
YSH_URL = 'https://sanpu.iarrp.cn/api/ssp/dcyd/xj/yzjsh/sl?xjdm=520327'
DSH_PAGE = 'https://sanpu.iarrp.cn/api/ssp/dcyd/xj/dzjsh/page?pageNum='
YSH_PAGE = 'https://sanpu.iarrp.cn/api/ssp/dcyd/xj/yzjsh/page?pageNum='
PAGE_TAIL = '&pageSize=50&xzqdm=520327'
BASE_NAME = os.getenv('BASE_NAME')
# headers info 
img_headers = {
    "Accept": os.getenv("ACCEPT"),
    "Accept-Encoding": os.getenv("ACCEPT_ENCODING"),
    "Accept-Language": os.getenv("ACCEPT_LANGUAGE"),
    "Authorization": os.getenv("AUTHORIZATION"),
    "Connection": os.getenv("CONNECTION"),
    "Content-Length": os.getenv("CONTENT_LENGTH"),
    "Content-Type": os.getenv("CONTENT_TYPE"),
    "Cookie": os.getenv("COOKIE"),
    "Host": os.getenv("HOST"),
    "origin": os.getenv("ORIGIN"),
    "Referer": os.getenv("REFERER"),
    "Sec-Ch-Ua": os.getenv("SEC_CH_UA"),
    "Sec-Ch-Ua-Mobile": os.getenv("SEC_CH_UA_MOBILE"),
    "Sec-Ch-Ua-Platform": os.getenv("SEC_CH_UA_PLATFORM"),
    "Sec-Fetch-Dest": os.getenv("SEC_FETCH_DEST"),
    "Sec-Fetch-Mode": os.getenv("SEC_FETCH_MODE"),
    "Sec-Fetch-Site": os.getenv("SEC_FETCH_SITE"),
    "User-Agent": os.getenv("USER_AGENT")
}
headers = {
    "Accept": os.getenv("ACCEPT"),
    "Accept-Encoding": os.getenv("ACCEPT_ENCODING"),
    "Accept-Language": os.getenv("ACCEPT_LANGUAGE"),
    "Authorization": os.getenv("AUTHORIZATION"),
    "Connection": os.getenv("CONNECTION"),
    "Cookie": os.getenv("COOKIE"),
    "Host": os.getenv("HOST"),
    "Referer": os.getenv("REFERER"),
    "Sec-Ch-Ua": os.getenv("SEC_CH_UA"),
    "Sec-Ch-Ua-Mobile": os.getenv("SEC_CH_UA_MOBILE"),
    "Sec-Ch-Ua-Platform": os.getenv("SEC_CH_UA_PLATFORM"),
    "Sec-Fetch-Dest": os.getenv("SEC_FETCH_DEST"),
    "Sec-Fetch-Mode": os.getenv("SEC_FETCH_MODE"),
    "Sec-Fetch-Site": os.getenv("SEC_FETCH_SITE"),
    "User-Agent": os.getenv("USER_AGENT")}


In [ ]:
### 辅助函数
# 装饰器，如果函数没有返回值，则一直重试
def retry_if_no_return(func):
    def wrapper(*args, **kwargs):
        # 定义重试次数
        retry_times = 5
        while retry_times>=0:
            result = func(*args, **kwargs)
            if result:
                return result
            # 如果函数没有返回值，可以在这里添加日志或等待时间
            time.sleep(2)
            retry_times -= 1
    return wrapper

# 遍历url列表下载图片
@retry_if_no_return
def download_image(url,save_path_name,headers):
    headers = headers
    try:
        r = requests.get(url, headers=headers)
        if r.status_code == 200:
            with open(save_path_name, 'wb') as f:
                f.write(r.content)
                return True
    except:
        return False
    time.sleep(random.random())

In [ ]:
# 获取施肥信息
def get_sf_info(point_id,headers):
    """获取施肥信息"""
    url = f"{SF_URL}{point_id}"
    # 发送GET请求
    response = requests.get(url, headers=headers)
    # 检查响应状态码
    if response.status_code == 200:
        # 解析响应内容
        data = response.json()
        return data
    else:
        print("请求失败，状态码：", response.status_code)
        return False
# 获取图片信息
def get_img_info(point_id,headers):
    """获取图片和视频链接"""
    url = IMG_URL
    # 请求的JSON数据
    # 请根据实际情况替换下面的data字典
    data = {
        "glbh": f"{int(point_id)}"
    }
    # 将字典转换为JSON格式的字符串
    payload = json.dumps(data)
    # 发送POST请求
    response = requests.post(url, headers=headers, data=payload)
    # 检查响应状态码
    if response.status_code == 200:
        # 解析响应内容
        data = response.json()
        return data
    else:
        print("请求失败，状态码：", response.status_code)
        return False
# 获取立地条件信息
def get_ldtj_info(point_id,headers):
    """获取立地条件信息"""
    url = f"{LDTJ_URL}{point_id}"
    # 发送GET请求
    response = requests.get(url, headers=headers)
    # 检查响应状态码
    if response.status_code == 200:
        # 解析响应内容
        data = response.json()
        return data
    else:
        print("请求失败，状态码：", response.status_code)
        return False
# 获取样点基本信息
def get_base_info(point_id,headers):
    """获取基本信息"""
    url = f"{JBXX_URL}{point_id}"
        # 发送GET请求
    response = requests.get(url, headers=headers)
    # 检查响应状态码
    if response.status_code == 200:
        # 解析响应内容
        data = response.json()
        return data
    else:
        print("请求失败，状态码：", response.status_code)
        return False
# 获取采土袋信息
def get_ctd_info(point_id,headers):
    """获取采土袋信息"""
    url = f"{CTD_URL}{point_id}"
    # 发送GET请求
    response = requests.get(url, headers=headers)
    # 检查响应状态码
    if response.status_code == 200:
        # 解析响应内容
        data = response.json()
        return data
    else:
        print("请求失败，状态码：", response.status_code)
        return False
    
# 获取剖面信息
def get_pm_info(point_id,headers):
    """获取剖面信息"""
    url = f"{PM_URL}{point_id}"
    # 发送GET请求
    response = requests.get(url, headers=headers)
    # 检查响应状态码
    if response.status_code == 200:
        # 解析响应内容
        data = response.json()
        return data
    else:
        print("请求失败，状态码：", response.status_code)
        return False
# 获取剖面发生层信息
def get_pmfc_info(point_id,headers):
    """获取剖面发生层信息"""
    url = f"{PM_FC_URL}{point_id}"
    # 发送GET请求
    response = requests.get(url, headers=headers)
    # 检查响应状态码
    if response.status_code == 200:
        # 解析响应内容
        data = response.json()
        return data
    else:
        print("请求失败，状态码：", response.status_code)
        return False
# 获取当前页面cookie信息
def get_cookie():
    """获取当前页面cookie信息"""
    # 使用JavaScript获取Cookie
    cookie_script = """
    return document.cookie;
    """
    cookie_value = driver.execute_script(cookie_script)
    return cookie_value
# 更新cookie信息
def update_cookie(headers):
    """更新cookie信息"""
    # 获取当前页面cookie信息
    cookie_value = get_cookie()
    # 更新headers
    headers.update({"Cookie": cookie_value})
    return headers
# 获取已审核数量
def get_audit_num(headers):
    """获取已审核数量"""
    url = YSH_URL
    # 发送GET请求
    response = requests.get(url, headers=headers)
    # 检查响应状态码
    if response.status_code == 200:
        # 解析响应内容
        data = response.json()
        return data['result']
    else:
        print("请求失败，状态码：", response.status_code)
        return False
# 获取待审核数量
def get_wait_num(headers):
    """获取待审核数量"""
    url = DSH_URL
    # 发送GET请求
    response = requests.get(url, headers=headers)
    # 检查响应状态码
    if response.status_code == 200:
        # 解析响应内容
        data = response.json()
        return data['result']
    else:
        print("请求失败，状态码：", response.status_code)
        return False
# 获取指定页面的编码
def get_set_page_num(url,headers):
    """获取指定页面的编码"""
    response = requests.get(url, headers=headers)
    # 检查响应状态码
    if response.status_code == 200:
        # 解析响应内容
        data = response.json()
        info_list = data['result']['records']
        # 循环获取编码
        temp_dict = [{_['ydbh']:_['ydlb']} for _ in info_list]
        return temp_dict
    else:
        print("请求失败，状态码：", response.status_code)
        return False

# 获取已审核的编码
def get_page_number(total,headers):
    """获取当前传入页面的编码,默认每一页50"""
    # 计算有多少页
    page_num = math.ceil(total / 50)
    # 循环获取每一页的编码
    result_list = []
    for i in range(1, page_num + 1):
        url = f"{YSH_PAGE}{i}{PAGE_TAIL}"
        result_list+=get_set_page_num(url,headers)
        time.sleep(random.random())
    return result_list

# 获取待审核的编码
def get_wait_page_number(total,headers):
    """获取当前传入页面的编码,默认每一页50"""
    # 计算有多少页
    page_num = math.ceil(total / 50)
    # 循环获取每一页的编码
    temp_list = []
    for i in range(1, page_num + 1):
        url = f"{DSH_PAGE}{i}{PAGE_TAIL}"
        temp_list+=get_set_page_num(url,headers)
        time.sleep(random.random())
    return temp_list
    
# 随机休眠
def random_sleep():
    """随机休眠"""
    time.sleep(random.random())
    return

In [ ]:
# 配置浏览器
# chrome.exe --remote-debugging-port=9999 --user-data-dir="D:\Selenium\AutomationProfile"
# chrome.exe --remote-debugging-port=9999 --user-data-dir="D:\Program Files\ChromeDir"
options = webdriver.ChromeOptions()
options.add_experimental_option("debuggerAddress", "localhost:9998")
driver = webdriver.Chrome(options=options)

driver.implicitly_wait(60)

In [ ]:
# 保存路径
save_path = r"F:\collection_spb_info\曾宇丽"
# 如果没有该文件夹，则创建
if not os.path.exists(save_path):
    os.makedirs(save_path)

# 检查heraders并更新

In [ ]:

headers = update_cookie(headers)

In [ ]:
headers

In [ ]:
# 获取当前账户下的待审核和已审核编码，并写入文件
# 更新headers
headers = update_cookie(headers)
audit_point_list = get_page_number(get_audit_num(headers),headers)

wait_point_list = get_wait_page_number(get_wait_num(headers),headers)
all_point = audit_point_list + wait_point_list
# 提取编码
loop_all_point = [list(item.keys())[0] for item in all_point]

In [ ]:
json_save_path = os.path.join(save_path,f'poind_id_{len(all_point)}.json')
# 写入JSON文件
with open(json_save_path, 'w') as json_file:
    json.dump(all_point, json_file, indent=4)
# 读取原始JSON文件
with open(json_save_path, 'r') as json_file:
    data_dict = json.load(json_file)

# 初始化字典，用于存储值为'1'的数据
data_with_1 = {}
# 遍历列表，每个元素都是一个字典
for item_dict in data_dict:
    for key, value in item_dict.items():
        if value == '0':
            pass
        elif value == '1':
            data_with_1[key] = value
# 存储值为剖面样的数据到JSON文件
with open(os.path.join(save_path,f'data_with_1_{len(data_with_1)}.json'), 'w') as json_file:
    json.dump(data_with_1, json_file, indent=4)


In [ ]:
# 读取原信息
point_check_info = pd.read_excel(os.path.join(save_path,'base_info.xlsx'))
point_check_list = list(point_check_info['ydbh'])

In [ ]:
# 获取更新点位
loop_all_point_update = []
loop_all_point_pmy = []
for one_check_point in all_point:
    check_id = list(one_check_point.keys())[0]
    check_type = list(one_check_point.values())[0]
    # 检查是否已经存在
    if int(check_id) in point_check_list:
        pass
    else:
        loop_all_point_update.append(check_id)
        if check_type == 1:
            loop_all_point_pmy.append(check_id)

In [ ]:
len(loop_all_point),len(loop_all_point_update),len(point_check_list),len(loop_all_point_pmy)

In [ ]:
len(all_point),len(wait_point_list),len(audit_point_list)

In [ ]:
# 更新替换信息
loop_all_point = loop_all_point_update

In [ ]:
# 测试一个点位的基本信息
base_info = get_base_info(loop_all_point[0],headers)
base_info

In [ ]:
# 获取基本信息
# 更新headers
headers = update_cookie(headers)
base_info_list = []
for one_point in tqdm(loop_all_point):
    one_point_base_info = get_base_info(one_point,headers)['result']
    # 随机休眠
    # time.sleep(random.random())
    base_info_list.append(one_point_base_info)


In [ ]:
df_base_info = pd.DataFrame(base_info_list)
df_base_info.head(1)

In [ ]:
# 保存基本信息
df_base_info.to_excel(os.path.join(save_path,f'base_info_update_{len(loop_all_point)}.xlsx'),index=False)

In [ ]:
# 测试一个点位的图片信息
img_info = get_img_info(loop_all_point[0],img_headers)
img_info

In [ ]:
# 获取图片链接信息
# 更新headers
img_headers = update_cookie(img_headers)
img_info_list = []
for one_point in tqdm(loop_all_point):
    one_point_img_info = get_img_info(one_point,img_headers)['result']
    # 为链接增加域名
    prefix = BASE_NAME
    updated_list = [{**item, 'wjlj': f"{prefix}{item['wjlj']}"} for item in one_point_img_info]
    # 随机休眠
    # time.sleep(random.random())
    img_info_list+=updated_list

In [ ]:
df_img_info = pd.DataFrame(img_info_list)
df_img_info.head(1)

In [ ]:
# 保存媒体信息
df_img_info.to_excel(os.path.join(save_path, f'img_info_update_{len(loop_all_point)}.xlsx'), index=False)

In [ ]:
# 测试一个点位的立地调查信息
ldtj_info = get_ldtj_info(loop_all_point[0],headers)
ldtj_info

In [ ]:
# 获取立地条件信息
# 更新headers
headers = update_cookie(headers)
ldtj_info_list = []
for one_point in tqdm(loop_all_point):
    one_point_ldtj_info = get_ldtj_info(one_point,headers)['result']
    ldtj_info_list.append(one_point_ldtj_info)
    # 随机休眠
    # time.sleep(random.random())

In [ ]:
df_ldtj_info = pd.DataFrame(ldtj_info_list)
df_ldtj_info.head(1)

In [ ]:
# 保存立地条件信息
df_ldtj_info.to_excel(os.path.join(save_path,f'ldtj_info_upate_{len(loop_all_point)}.xlsx'),index=False)

In [ ]:
# 测试一个点位的采土袋信息
ctd_info = get_ctd_info(loop_all_point[0],headers)
ctd_info

In [ ]:
# 获取采土袋信息
# 更新headers
headers = update_cookie(headers)
ctd_info_list = []
for one_point in tqdm(loop_all_point):
    one_point_ctd_info = get_ctd_info(one_point,headers)['result']
    ctd_info_list+=one_point_ctd_info
    # 随机休眠
    # time.sleep(random.random())

In [ ]:
df_ctd_info = pd.DataFrame(ctd_info_list)
df_ctd_info.head(1)

In [ ]:
# 保存采土袋信息
df_ctd_info.to_excel(os.path.join(save_path, f'ctd_info_update_{len(loop_all_point)}.xlsx'), index=False)

In [ ]:
# 测试一个点位的施肥信息
sf_info = get_sf_info(loop_all_point[0],headers)
sf_info

In [ ]:
# 获取施肥信息
# 更新headers
headers = update_cookie(headers)
sf_info_list = []
for one_point in tqdm(loop_all_point):
    one_point_sf_info = get_sf_info(one_point,headers)['result']
    sf_info_list+=one_point_sf_info
    # 随机休眠
    # time.sleep(random.random())

In [ ]:
df_sf_info = pd.DataFrame(sf_info_list)
df_sf_info.head(1)

In [ ]:
# 保存施肥信息
df_sf_info.to_excel(os.path.join(save_path, f'sf_info_update_{len(loop_all_point)}.xlsx'), index=False)

# 剖面信息

In [ ]:
# 剖面点位信息
# 读取原始JSON文件
with open(os.path.join(save_path,f'data_with_1_{len(data_with_1)}.json'), 'r') as json_file:
    pm_dict = json.load(json_file)
pm_point_id = [_ for _,__ in pm_dict.items()]


In [ ]:
# 测试剖面点位的信息
pm_info = get_pm_info(pm_point_id[0],headers)
pm_info

In [ ]:
# 获取剖面信息
# 更新headers
headers = update_cookie(headers)
pm_info_list = []
for one_point in tqdm(pm_point_id):
    one_point_pm_info = get_pm_info(one_point,headers)['result']
    pm_info_list.append(one_point_pm_info)
    # 随机休眠
    time.sleep(random.random())

In [ ]:
df_pm_info = pd.DataFrame(pm_info_list)
df_pm_info.head(1)

In [ ]:
# 保存剖面信息
df_pm_info.to_excel(os.path.join(save_path, 'pm_info.xlsx'), index=False)

In [ ]:
# 测试剖面点位发生层信息
pm_fc_info = get_pmfc_info(pm_point_id[0],headers)
pm_fc_info

In [ ]:
# 获取剖面发生层信息
# 更新headers
headers = update_cookie(headers)
pm_fc_info_list = []
for one_point in tqdm(pm_point_id):
    one_point_pm_fc_info = get_pmfc_info(one_point,headers)['result']
    pm_fc_info_list+=one_point_pm_fc_info
    # 随机休眠
    time.sleep(random.random())

In [ ]:
df_pm_fc_info = pd.DataFrame(pm_fc_info_list)
df_pm_fc_info.head(1)

In [ ]:
# 保存剖面发生层信息
df_pm_fc_info.to_excel(os.path.join(save_path, 'pm_fc_info.xlsx'), index=False)

# TODO

In [ ]:
# 合并更新后的信息表和原始表
excel_name_list = set([_.split('_')[0] for _ in  os.listdir(save_path) if _.endswith('.xlsx')])

In [ ]:

for one_table in tqdm(excel_name_list):
    # 初始化空DataFrame
    result_df = pd.DataFrame()
    temp_df_list = []
    for one_excel in [_ for _ in os.listdir(save_path) if _.endswith('.xlsx')]:
        if one_excel.split('_')[0] == one_table:
            temp_df_list.append(pd.read_excel(os.path.join(save_path, one_excel)))
    result_df = pd.concat(temp_df_list)
    # 转换内容为字符串
    result_df = result_df.astype(str)
    result_df.to_excel(os.path.join(save_path, one_table + '_info.xlsx'),index=False)